In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv')
train_df.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [4]:
test_df = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv')
test_df.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32


In [5]:
train_df.drop(['id'], axis=1, inplace = True)

In [6]:
all_cols = train_df.columns.tolist()
num_cols = [
            'Previous qualification (grade)', 'Admission grade', 'Age at enrollment',
            'Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)',
            'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)',
            'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)',
            'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 
            'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)',
            'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)',
            'Unemployment rate', 'Inflation rate', 'GDP'
           ]
cat_cols = [col for col in all_cols if col not in num_cols and col != 'Target']
train_df[cat_cols] = train_df[cat_cols].astype('object')

In [7]:
subset_train_df = train_df[train_df['Target'].isin(['Dropout', 'Graduate'])]

In [8]:
subset_train_df.shape

(61578, 37)

In [9]:
subset_train_df.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
5,1,39,1,171,1,19,133.1,1,19,19,...,0,0,0,0,0.000000,0,13.9,-0.3,0.79,Dropout


In [10]:
def frequency_encode_df(train_df, cat_cols):
    for col in cat_cols:
        freq_encoding = train_df[col].value_counts() / len(train_df)
        train_df[col] = train_df[col].map(freq_encoding)
    
    return train_df

In [11]:
encoded_sub = frequency_encode_df(subset_train_df, cat_cols)


In [12]:
encoded_sub.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.9119,0.456738,0.701192,0.111127,0.911202,0.869710,126.0,0.993293,0.262188,0.240638,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,0.9119,0.211196,0.701192,0.111127,0.911202,0.869710,125.0,0.993293,0.243090,0.240638,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,0.9119,0.211196,0.116097,0.065624,0.911202,0.869710,137.0,0.993293,0.073500,0.240638,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
4,0.9119,0.456738,0.116097,0.177953,0.911202,0.869710,132.0,0.993293,0.243090,0.308292,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
5,0.9119,0.202767,0.701192,0.043051,0.911202,0.044074,133.1,0.993293,0.243090,0.240638,...,0,0,0,0,0.000000,0,13.9,-0.3,0.79,Dropout


In [13]:
X = encoded_sub.drop(['Target'], axis =1)
y = encoded_sub['Target']

In [14]:
X.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,0.9119,0.456738,0.701192,0.111127,0.911202,0.869710,126.0,0.993293,0.262188,0.240638,...,0,0,6,7,6,12.428571,0,11.1,0.6,2.02
1,0.9119,0.211196,0.701192,0.111127,0.911202,0.869710,125.0,0.993293,0.243090,0.240638,...,0,0,6,9,0,0.000000,0,11.1,0.6,2.02
2,0.9119,0.211196,0.116097,0.065624,0.911202,0.869710,137.0,0.993293,0.073500,0.240638,...,0,0,6,0,0,0.000000,0,16.2,0.3,-0.92
4,0.9119,0.456738,0.116097,0.177953,0.911202,0.869710,132.0,0.993293,0.243090,0.308292,...,0,0,7,12,6,12.933333,0,7.6,2.6,0.32
5,0.9119,0.202767,0.701192,0.043051,0.911202,0.044074,133.1,0.993293,0.243090,0.240638,...,0,0,0,0,0,0.000000,0,13.9,-0.3,0.79


In [15]:
y.head()

0    Graduate
1     Dropout
2     Dropout
4    Graduate
5     Dropout
Name: Target, dtype: object

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=80, random_state=42, max_depth=8)
model.fit(X_train, y_train)
y_prob =model.predict_proba(X_test)
threshold = 0.8
y_pred = ['Enrolled' if max(probs) < threshold else model.classes_[np.argmax(probs)] for probs in y_prob]

accuracy = accuracy_score(y_test, y_pred)
print(f"overall Accuracy: {accuracy:.4f}")

overall Accuracy: 0.8723
